In [1]:
from appgeopy import *
from mgtwr.function import _compute_betas_gwr
from mgtwr.kernel import GTWRKernel
from mgtwr.model import GTWR, MGTWR
from mgtwr.sel import SearchGTWRParameter, SearchMGTWRParameter
from my_packages import *

In [2]:
input_df = pd.read_pickle("mgtwr_dset_MLCW_L1.xz")
input_df.head(5)

,X_TWD97,Y_TWD97,time,MLCW_L1,disp,electricity,gwl_L1,gwl_L2D,gwl_L2S,gwl_L3,gwl_L4,rainfall
0,179539.204623,2.602035e+06,0,-1.0,-1.293143,-0.491677,-0.818277,-1.352586,-0.938242,-0.637636,-1.063519,-1.417241
1,178859.958807,2.608229e+06,0,-1.0,-1.538345,0.070866,-0.763357,-1.330307,-1.120232,-0.593373,-0.640301,-1.523101
2,173088.151033,2.608157e+06,0,-5.0,-2.175268,-0.062627,-0.832466,-1.242687,-0.976948,-0.666550,-0.490628,-0.873834
3,175783.144962,2.616755e+06,0,-11.0,-1.814172,0.496965,-0.687335,-0.842182,-1.082041,-0.499370,0.049202,-1.116128
4,190429.148778,2.629865e+06,0,-1.0,-1.027422,-0.540083,0.036527,0.194170,0.320681,0.066678,0.400741,-1.564612


In [3]:
# Prepare coordinates and time
space_coords = input_df[["X_TWD97", "Y_TWD97"]].values
time_coords = input_df[["time"]].values

response_var = input_df["MLCW_L1"]
predictor_vars = input_df.loc[:, ["gwl_L1", "gwl_L2S", "gwl_L2D","gwl_L3","gwl_L4"]]
predictor_vars

,gwl_L1,gwl_L2S,gwl_L2D,gwl_L3,gwl_L4
0,-0.818277,-0.938242,-1.352586,-0.637636,-1.063519
1,-0.763357,-1.120232,-1.330307,-0.593373,-0.640301
2,-0.832466,-0.976948,-1.242687,-0.666550,-0.490628
3,-0.687335,-1.082041,-0.842182,-0.499370,0.049202
4,0.036527,0.320681,0.194170,0.066678,0.400741
...,...,...,...,...,...
2171,0.299286,-0.088361,-0.532607,-0.096380,0.174399
2172,1.033645,0.112993,-0.185650,-0.071813,-0.000209
2173,0.977751,0.725844,1.070821,0.326752,-0.604508
2174,-0.643542,-1.000723,-1.108507,-0.195308,0.314285


In [4]:
# Start execution timer for GTWR bandwidth optimization process
start = time.time()

# Initialize SearchGTWRParameter for bandwidth optimization
# - space_coords: Spatial coordinates matrix (n x 2)
# - time_coords: Temporal coordinates vector (n x 1)
# - predictor_vars: Independent variables matrix (n x k)
# - response_var: Dependent variable vector (n x 1)
# - kernel="gaussian": Weight observations using Gaussian decay function
# - fixed=True: Use fixed distance-based bandwidth (vs. adaptive)
# - thread=4: Parallelize computation across 4 CPU threads
sel = SearchGTWRParameter(
    coords=space_coords,
    t=time_coords,
    X=predictor_vars,
    y=response_var,
    kernel="gaussian",
    fixed=True,
    thread=8,
)

# Execute bandwidth optimization using golden section search algorithm
# - tau_max=np.max(time_coords): Set maximum temporal bandwidth to data range
# - verbose=True: Display iteration progress
# - time_cost=True: Report execution time
# - max_iter=100: Limit optimization iterations to prevent excessive runtime
# - bw_decimal/tau_decimal: Control precision of spatial/temporal bandwidths
spatial_bw, temporal_bw = sel.search(
    criterion="AICc",
    tau_min=0,
    tau_max=12,
    verbose=True,
    time_cost=True,
    max_iter=500,
    bw_decimal=3,
    tau_decimal=0,
    tol=1e-20,
)

# Output optimized bandwidth parameters
print(spatial_bw, temporal_bw)

# Calculate and format total execution time in hours:minutes:seconds
end = time.time()
elapsed_time = end - start
hours = int(elapsed_time // 3600)
minutes = int((elapsed_time % 3600) // 60)
seconds = elapsed_time % 60
print("Processing time: {} hours {} minutes {:.2f} seconds".format(hours, minutes, seconds))

bw:  21392.42 , tau:  11.0 , score:  9948.668213090694
bw:  13759.135 , tau:  11.0 , score:  9888.353058244387
bw:  9041.715 , tau:  11.0 , score:  9834.119829702237
bw:  6126.029 , tau:  11.0 , score:  9802.231010657672
bw:  4324.116 , tau:  11.0 , score:  9785.101286025852
bw:  3210.411 , tau:  11.0 , score:  9777.947559154376
bw:  2522.135 , tau:  11.0 , score:  9774.748252328118
bw:  2096.733 , tau:  11.0 , score:  9774.505724827459
bw:  2096.733 , tau:  11.0 , score:  9774.505724827459
bw:  2259.224 , tau:  11.0 , score:  9774.213414654816
bw:  2259.224 , tau:  11.0 , score:  9774.213414654816
bw:  2259.224 , tau:  11.0 , score:  9774.213414654816
bw:  2259.224 , tau:  11.0 , score:  9774.213414654816
bw:  2259.224 , tau:  11.0 , score:  9774.213414654816
bw:  2259.224 , tau:  11.0 , score:  9774.213414654816
bw:  2250.166 , tau:  11.0 , score:  9774.212482440214
bw:  2250.166 , tau:  11.0 , score:  9774.212482440214
bw:  2250.166 , tau:  11.0 , score:  9774.212482440214
bw:  2250

In [5]:
gtwr = GTWR(
    space_coords,
    time_coords,
    predictor_vars,
    response_var,
    spatial_bw,
    temporal_bw,
    kernel="gaussian",
    fixed=True,
).fit()

In [6]:
gtwr.adj_R2, gtwr.R2

(0.25886717697212835, 0.3102685483408115)

In [7]:
start = time.time()

sel_multi = SearchMGTWRParameter(
    coords=space_coords,
    t=time_coords,
    X=predictor_vars,
    y=response_var,
    kernel="gaussian",
    fixed=True,
    thread=2,
)

bws = sel_multi.search(
    criterion="AICc",
    tol=1e-5,
    tol_multi=1.0e-5,
    init_bw=spatial_bw,
    init_tau=1,
    time_cost=True,
    bw_decimal=3,
    tau_decimal=0,
    verbose=True,
    rss_score=True,
)

# Calculate and format total execution time in hours:minutes:seconds
end = time.time()
elapsed_time = end - start
hours = int(elapsed_time // 3600)
minutes = int((elapsed_time % 3600) // 60)
seconds = elapsed_time % 60
print("Processing time: {} hours {} minutes {:.2f} seconds".format(hours, minutes, seconds))

Current iteration: 1 ,SOC: 0.5968892
Bandwidths: 5274.212, 1644.668, 1487.342, 2290.385, 2063.362, 4492.41
taus: 2.0,2.0,2.0,2.0,2.0,2.0
Current iteration: 2 ,SOC: 0.0439356
Bandwidths: 5297.543, 3311.837, 1352.845, 2582.202, 2630.974, 6030.562
taus: 2.0,2.0,2.0,2.0,2.0,2.0
Current iteration: 3 ,SOC: 0.0175202
Bandwidths: 5524.07, 4564.514, 1332.912, 2805.302, 3057.266, 6882.4
taus: 2.0,2.0,2.0,2.0,2.0,2.0
Current iteration: 4 ,SOC: 0.0097941
Bandwidths: 5561.824, 5649.653, 1327.405, 3038.648, 3405.153, 7673.12
taus: 2.0,2.0,2.0,2.0,2.0,2.0
Current iteration: 5 ,SOC: 0.0041303
Bandwidths: 5565.232, 6733.477, 1328.206, 3324.149, 3825.972, 8204.987
taus: 2.0,2.0,2.0,2.0,2.0,2.0
Current iteration: 6 ,SOC: 0.0003726
Bandwidths: 5561.824, 7992.947, 1327.405, 3729.247, 4535.667, 8350.534
taus: 2.0,2.0,2.0,2.0,2.0,2.0
Current iteration: 7 ,SOC: 0.0032153
Bandwidths: 5559.72, 9667.763, 1328.703, 4463.566, 5946.169, 8054.009
taus: 2.0,2.0,2.0,2.0,2.0,2.0
Current iteration: 8 ,SOC: 0.007592
Band

In [8]:
mgtwr = MGTWR(
    coords=space_coords,
    t=time_coords,
    X=predictor_vars,
    y=response_var,
    selector=sel_multi,
    kernel="gaussian",
    fixed=True,
    thread=8,
).fit()
print(mgtwr.R2)

0.262520403300586
